# La gestion des modules en Python

## Module ou *package* ?

**Module :** fichier qui regroupe plusieurs fonctions.

**Package :** répertoire qui rassemble plusieurs modules.

Dans le précédent calepin, nous avons calculé une régression linéaire avec la méthode des moindres carrés. De cet énoncé nous ressortons deux implicites :

- Nous pourrions utiliser un autre modèle ;
- la méthode des moindres carrés n’est pas la seule possible.

Nous aurions effectivement pu décider de réaliser une régression polynomiale ou de calculer une descente de gradient. De là, s’il fallait consigner dans un même script toutes les fonctions nécessaires sans jamais les rendre disponibles pour d’autres programmes, ils feraient rapidement tous des dizaines de milliers de lignes.

Heureusement, ce n’est pas le cas et plutôt que de réécrire la constante $\pi$ à chaque fois que nous avons besoin de faire des calculs dans le cercle, il nous suffit de l’importer depuis le module *math* :

In [ ]:
import math

def circle_area(r):
    return math.pi * r ** 2

## Importations et espaces de nommage

Dans le répertoire *scripts* se trouve un sous-répertoire *dummy* avec un script *linear.py* qui contient une fonction pour générer des données à l’aspect linéaire. Si nous souhaitons l’importer, nous pouvons l’appeler par son nom :

In [ ]:
import linear

**Comment ça ?!** `ModuleNotFoundError` ?

Python a raison, il s’attendait à trouver un fichier *linear.py* au même niveau que ce calepin, c’est-à-dire dans le même répertoire, alors qu’il est rangé dans le répertoire *./scripts/dummy/*.

Le sous-répertoire *dummy* est ce que l’on appelle un *package* et un *package* peut renfermer plusieurs modules, *linear* étant l’un d’eux.

### Importer un module depuis un paquet

Repartons à présent du squelette de notre script *linear_regression.py* disponible dans le répertoire *scripts* afin d’avoir directement accès au paquet *dummy* et ajoutons une instruction pour l’importer :

```python
import dummy
```

On pourrait supposer que cette commande donne accès à tous les modules qui dépendent de `dummy` mais, en l’état, aucune possibilité d’accéder à la fonction `generate_data()` définie dans `linear.py`. Toutes les commandes suivantes échouent :

```python
data = generate_data()
data = linear.generate_data()
data = dummy.linear.generate_data()
```

Une solution consiste à importer directement le module `linear` du *package* `dummy` :

```python
import dummy.linear
```

Puis à appeler la fonction en passant par tous les espaces de nommage :

```python
data = dummy.linear.generate_data()
```

### Définir un alias

Plutôt fastidieux d’appeler une fonction d’aussi loin, non ? Heureusement, il existe les alias :

```python
import dummy.linear as dl

data = dl.generate_data()
```

### Importation complète

On peut aussi accéder directement aux fonctions sans les espaces de nommage :

```python
from dummy.linear import *

data = generate_data()
```

### Importation spécifique

L’importation totale des fonctions d’un module présente deux risques :

- Charger des fonctions inutiles pour le script en cours ;
- l’étoile `*` masque le nom des fonctions.

Une autre méthode consiste à puiser une fonction spécifique dans un module :

```python
from dummy.linear import generate_data

data = generate_data()
```

## Organiser son *package*

Dans la pratique, on préférera toujours conserver dans la syntaxe d’appel aux fonctions utilisateur le lien de dépendance à un espace de nommage (module) :

```python
from dummy import linear

data = linear.generate_data()
```

De cette manière, si l’on définissait dans un module une fonction qui porte le même nom qu’une autre fonction d’un autre module, on éviterait tout risque de collision.

### Des fichiers thématiques

Un *package* s’organise autour de fichiers regroupant des fonctions qui servent elles-mêmes un objectif commun. On peut imaginer des modules avec des noms comme : `core`, `utils`, `tests`, `data`…

### Un sommaire pour le paquet

Il existe un fichier un peu particulier que l’on a coutume de placer en-tête de tout paquet, et ce même s’il est devenu optionnel avec les mises à jour successives de python : `__init__.py`

Ce fichier est utilisé à l’initialisation du paquet et peut se concevoir comme un sommaire du contenu. Il est chargé de lister l’ensemble des modules qu’il contient.

Créez un fichier `__init__.py` à la racine du répertoire `dummy` avec juste ces quelques lignes :

```python
__name__ = 'Dummy: the package that generate dummy data'
__version__ = 'v1.0b'

from .linear import generate_data
```

Ce qui permet ensuite de raccourcir les appels aux fonctions de tous les modules dépendants, comme s’il s’agissait de méthodes du paquet :

```python
import dummy

data = dummy.generate_data()
```

## Les variables spéciales : l’exemple de `__name__`

Il existe des variables un peu particulières encadrées par des doubles *underscores* :

- `__name__`
- `__version__`

Ce sont des variables spéciales qui permettent d’indiquer certaines informations à Python. Elles sont facultatives et ne servent que dans des cas particuliers.

Prenons l’exemple de la variable `__name__` que l’on trouve dans l’expression :

```python
if __name__ == '__main__':
    # Main program
```

Lorsque Python lit un fichier `.py`, il réalise deux opérations :

- exécuter une instruction similaire à : `__name__ = '__main__'`
- exécuter le code du script

D’un point du vue pratique, observons le fichier [*linear.py*](scripts/dummy/linear.py), qui dispose d’un morceau de code :

```python
if __name__ == '__main__':
    """Usage example.
    Works only when used as standalone.
    """
    import matplotlib.pyplot as plt

    # generate data
    X, Y = generate_data(size=100)

    # make plot
    coef = np.polyfit(X, Y, 1)
    poly1d_fn = np.poly1d(coef)
    plt.plot(X, Y, 'yo', X, poly1d_fn(X), '--k')

    # display plot
    plt.show()
```

Ce morceau de code n’est exécuté que lorsque le script *linear.py* est appelé tout seul (*standalone*). Ouvrez une nouvelle fenêtre de terminal, déplacez-vous dans le répertoire où se trouve le script et lancez-le :

```shell
$ python ./linear.py
```

Une fenêtre s’ouvrira dans votre interface avec un graphique représentant une régression linéaire.

En revanche, lorsque vous importez le module depuis un script, le graphique ne s’affiche pas !

```python
import dummy.linear
```

L’explication est simple : lorsque le script est importé en tant que module, son `__name__` prend la valeur du fichier. En l’occurrence, le script *linear.py* a pour nom `linear` et pas `__main__`.

Le nom `__main__` est réservé au script principal, à savoir le script *linear_regression.py* depuis lequel vous exécutez l’importation !

Si nous retirons du fichier *linear.py* la ligne contenant l’instruction `if __name__ = '__main__':` et que nous relançons l’exécution du script `linear_regression.py`, cette fois-ci le graphique apparaîtra !

La condition `if __name__ = '__main__':` est par conséquent impérative pour éviter tout effet de bord dans un script.